In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
# Tạo thư mục lưu biểu đồ
os.makedirs("charts", exist_ok=True)

# Đọc dữ liệu
df = pd.read_csv("clean_data_P2.csv")

In [ ]:
# Chuẩn hóa dữ liệu

# Min-Max cho Giá và Diện tích
scaler_minmax = MinMaxScaler()
df[["Giá_MinMax", "Diện tích_MinMax"]] = scaler_minmax.fit_transform(
df[["Giá (VND)", "Diện tích (m2)"]]
)

# Z-score cho Giá
scaler_zscore = StandardScaler()
df["Giá_Zscore"] = scaler_zscore.fit_transform(df[["Giá (VND)"]])

# Lưu DataFrame đã chuẩn hóa ra file CSV mới
df.to_csv("clean_data_P3.csv", index=False, encoding="utf-8-sig")
from google.colab import files
files.download("clean_data_P3.csv")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Vẽ biểu đồ
# Histogram: phân phối giá (đơn vị: Tỷ VND)
plt.figure(figsize=(8, 5))
sns.histplot(df["Giá (VND)"] / 1e9, bins=50, kde=True)
plt.title("Phân phối Giá nhà")
plt.xlabel("Giá (Tỷ VND)")
plt.ylabel("Số lượng")
plt.savefig("charts/histogram_gia.png")
plt.close()


# Boxplot: so sánh giá theo loại hình căn hộ (đơn vị Tỷ VND)
plt.figure(figsize=(12, 5))  # rộng hơn, thấp hơn
sns.boxplot(x="Loại hình căn hộ", y=df["Giá (VND)"] / 1e9, data=df)
plt.title("So sánh Giá theo Loại hình căn hộ")
plt.xlabel("Loại hình căn hộ")
plt.ylabel("Giá (Tỷ VND)")
plt.xticks(rotation=30, ha="right")  # xoay nhãn X cho dễ đọc
plt.tight_layout()
plt.savefig("charts/boxplot_gia_loaihinh.png")
plt.close()


# Scatter Plot: diện tích và giá (đơn vị tỷ VND)
plt.figure(figsize=(8, 6))
sns.scatterplot(x="Diện tích (m2)", y=df["Giá (VND)"] / 1e9, data=df)
plt.title("Mối quan hệ giữa Diện tích và Giá")
plt.xlabel("Diện tích (m2)")
plt.ylabel("Giá (Tỷ VND)")
plt.savefig("charts/scatter_dientich_gia.png")
plt.close()


# Bar Chart: số lượng tin đăng theo quận/huyện
df["Quận/Huyện"] = df["Địa chỉ"].apply(     # Tách tên quận/huyện từ địa chỉ
    lambda x: str(x).split(",")[-2].strip() if pd.notnull(x) and "," in str(x) else x
)
plt.figure(figsize=(12, 6))                 # Vẽ Bar Chart theo quận/huyện
df["Quận/Huyện"].value_counts().head(10).plot(kind="bar")
plt.title("Top 10 Quận/Huyện có nhiều tin đăng nhất")
plt.xlabel("Quận/Huyện")
plt.ylabel("Số lượng tin")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("charts/bar_tindang_quan.png")
plt.close()


# Pie Chart: tỷ lệ loại hình căn hộ
plt.figure(figsize=(5, 5))
counts = df["Loại hình căn hộ"].value_counts()
labels = counts.index.tolist()
labels = [                                  # Đảo lại vị trí "Đất" và "Văn phòng, Mặt bằng kinh doanh"
    "Nhà ở" if "Nhà ở" in labels else labels[0],
    "Căn hộ/Chung cư" if "Căn hộ/Chung cư" in labels else labels[1],
    "Văn phòng, Mặt bằng kinh doanh" if "Văn phòng, Mặt bằng kinh doanh" in labels else labels[2],
    "Đất" if "Đất" in labels else labels[3],
]
counts.plot(                                # Vẽ pie chart với labels đã sắp xếp
    kind="pie",
    labels=labels,
    autopct="%1.1f%%",
    startangle=90,
    textprops={'fontsize': 8}
)
plt.title("Tỷ lệ loại hình căn hộ")
plt.ylabel("")
plt.tight_layout()
plt.savefig("charts/pie_loaihinh.png")
plt.close()


In [ ]:
# Nén file charts chứa biểu đồ thành file zip
import shutil
shutil.make_archive("charts", "zip", "charts")
# Tải file zip về
from google.colab import files
files.download("charts.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Kết luận ban đầu:
# Phân phối giá nhà lệch phải, nhiều tin đăng ở mức giá thấp, một số ít ở giá rất cao.
# Boxplot cho thấy căn hộ chung cư và đất nền có khoảng giá khác biệt rõ rệt.
# Scatter plot cho thấy xu hướng diện tích càng lớn thì giá càng tăng, nhưng không tuyến tính.
# Một số quận tập trung nhiều tin đăng hơn hẳn, thể hiện thị trường sôi động.
# Loại hình căn hộ chủ yếu tập trung vào 2-3 loại chính, các loại khác chiếm tỷ lệ nhỏ.